In [125]:
import pandas as pd

df = pd.read_csv("../data/train.csv")
print(len(df))

2800


# Noise

### 자주 쓰이는 한자 탐색

In [126]:
import pandas as pd
import re
from collections import Counter

# 한자 추출 함수 정의
def extract_hanza(text):
    # CJK 통합 한자 범위(U+4E00 ~ U+9FFF)에 해당하는 문자 추출
    return "".join(re.findall("[\u4e00-\u9fff]", text))

# 문자 빈도 축정
def get_char_frequency(text_list):
    # 모든 문자를 하나의 리스트로 합치기
    chars = [char for text in text_list for char in text]
    # Counter를 사용하여 빈도 계산
    char_counts = Counter(chars)
    # 빈도순으로 정렬
    return pd.Series(char_counts).sort_values(ascending=False)

In [127]:
# 한자 빈도수 측정
df["hanza"] = df["text"].apply(extract_hanza)

char_frequency = get_char_frequency(df["hanza"])
filtered_chars = char_frequency[char_frequency >= 2]

zazu_hanza = list(filtered_chars.keys())
print(len(zazu_hanza), zazu_hanza)

23 ['美', '北', '中', '朴', '靑', '日', '與', '文', '英', '野', '佛', '伊', '獨', '反', '前', '軍', '硏', '對', '外', '社', '黃', '亞', '韓']


In [128]:
df_hanza = df[df["hanza"].str.len()!=0]
print(len(df_hanza))
df_hanza = df_hanza[["ID", "text", "target"]]
df_hanza.to_csv("hanza.csv",index=False)
df_hanza.head()

313


,ID,text,target
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0
8,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6
50,ynat-v1_train_00050,"美MBA[여성 비율x계속 x가4주$E19a대 U입생 중Ym,%",6
58,ynat-v1_train_00058,한#M2 !는 유`8 치료제 오B솔 美 임{ 3a 본격화,6


### 한자 전부 제거

In [129]:
def remove_chars_from_text(df, char_list):
    # 모든 문자를 정규 표현식 패턴으로 변환
    pattern = "[" + re.escape("".join(char_list)) + "]"

    # 'text' 열에서 해당 패턴의 문자 제거
    df["filtered_text"] = df["filtered_text"].str.replace(pattern, "", regex=True)
    return df

In [130]:
df["filtered_text"] = df["text"].copy()
df = remove_chars_from_text(df, zazu_hanza)
df = df.drop(columns=["hanza"])
df.head()

,ID,text,target,filtered_text
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,K찰.국DLwo 로L3한N% 회장 2 T0&}송=
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,"m 김정) 자주통일 새,?r열1나가야1보"
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,pI대선I앞두고 R2fr단 발] $비해 감시 강화


### 자주 쓰이는 특수문자 탐색

In [131]:
#영어, 숫자, 한글 제외
def extract_special(text):
    # CJK 통합 한자 범위(U+4E00 ~ U+9FFF)에 해당하는 문자 추출
    return "".join(re.findall(r"[^a-zA-Z0-9\sㄱ-ㅎㅏ-ㅣ가-힣]", text))

In [132]:
# 특수문자 빈도수 측정
special = df["filtered_text"].apply(extract_special)
char_frequency = get_char_frequency(special)
filtered_chars = char_frequency[char_frequency >= 100]
zazu_special = list(filtered_chars.keys())
print(len(zazu_special),zazu_special)

34 ['…', '.', '·', '%', '"', '-', '(', '|', '?', ',', '}', ':', '&', '_', '{', '~', '#', '\\', '*', ')', '$', ';', '`', '=', '+', "'", '!', '@', '<', '/', '>', '[', ']', '^']


### 100번 이상 사용된 특수문자 제거

In [133]:
def remove_chars_from_text(df, char_list):
    # 모든 문자를 정규 표현식 패턴으로 변환
    pattern = "[" + re.escape("".join(char_list)) + "]"

    # 'text' 열에서 해당 패턴의 문자 제거
    df["filtered_text"] = df["filtered_text"].str.replace(pattern, "", regex=True)
    return df

In [134]:
df = remove_chars_from_text(df, zazu_special)
df["filtered_list"] = df["filtered_text"].apply(str.split)
df.head()

,ID,text,target,filtered_text,filtered_list
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,정i 파1 미사z KT 이용기간 2e 단 Q분종U2보,"[정i, 파1, 미사z, KT, 이용기간, 2e, 단, Q분종U2보]"
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,K찰국DLwo 로L3한N 회장 2 T0송,"[K찰국DLwo, 로L3한N, 회장, 2, T0송]"
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,m 김정 자주통일 새r열1나가야1보,"[m, 김정, 자주통일, 새r열1나가야1보]"
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩,"[갤노트8, 주말, 27만대, 개통시장은, 불법, 보조금, 얼룩]"
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,pI대선I앞두고 R2fr단 발 비해 감시 강화,"[pI대선I앞두고, R2fr단, 발, 비해, 감시, 강화]"


### 형태소 분석기를 통한 한글+영문+숫자+특문 혼합 단어 탐지

In [135]:
# !pip install KoNLPy

In [136]:
# 분석기 임포트
from konlpy.tag import *

# 각 분석기 객체 생성
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [137]:
df["hannanum"] = df["filtered_list"].apply(lambda li: [hannanum.pos(e) for e in li])
df["kkma"] = df["filtered_list"].apply(lambda li: [kkma.pos(e) for e in li])
df["komoran"] = df["filtered_list"].apply(lambda li: [komoran.pos(e) for e in li])
df["okt"] = df["filtered_list"].apply(lambda li: [okt.pos(e) for e in li])

In [138]:
from collections import Counter


def check_tags_count(morpheme_lists):
    # 태그 카운트
    all_tags = [tag for sublist in morpheme_lists for _, tag in sublist]
    tag_counts = Counter(all_tags)

    alpha = tag_counts["Alpha"]
    punctuation = tag_counts["Punctuation"]
    number = tag_counts["Number"]
    rest = sum(tag_counts.values()) - (alpha + punctuation +number)

    # 조건 리스트
    conditions = []
    conditions.append(alpha >= 1 and punctuation >= 1 and number >= 1)
    conditions.append(alpha >= 2)
    conditions.append(punctuation >= 2)
    conditions.append(number >= 2)

    return any(conditions)

# 데이터프레임에 적용
df["check"] = df["okt"].apply(check_tags_count)

In [139]:
df_morph_condition = df[df["check"]==True]
df_morph_condition = df_morph_condition[["ID","target","text"]]
df_morph_condition.to_csv("df_morph_condition.csv",index=False)
print(len(df_morph_condition))
df_morph_condition.head()

1777


,ID,target,text
0,ynat-v1_train_00000,4,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
1,ynat-v1_train_00001,3,K찰.국DLwo 로L3한N% 회장 2 T0&}송=
2,ynat-v1_train_00002,2,"m 김정) 자주통일 새,?r열1나가야1보"
3,ynat-v1_train_00003,5,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩
4,ynat-v1_train_00004,6,pI美대선I앞두고 R2fr단 발] $비해 감시 강화


### 한글 샌드위치 패턴

In [33]:
# def detect_sandwich_pattern(text):
#     pattern = r"[가-힣]+[^가-힣]+[가-힣]+"
#     return bool(re.search(pattern, text))

def detect_sandwich_pattern(text_list):
    return any(
        bool(re.search(r"[가-힣]+[^가-힣]+[가-힣]+", str(item))) for item in text_list
    )

df["sandwich"] = df["filtered_list"].apply(detect_sandwich_pattern)
sandwich_df = df[df["sandwich"]==True]
not_sandwich_df = df[df["sandwich"] == False]

In [34]:
print(len(sandwich_df),len(not_sandwich_df))

2069 731


In [35]:
not_sandwich_df = not_sandwich_df[["ID", "target", "text"]]
not_sandwich_df.to_csv("not_sandwich_df.csv", index=False)

### 특수문자 기반 노이즈 탐색

In [ ]:
import re

# 특수문자 기반의 노이즈 탐색 
def calculate_noise_ratio(df):
    df["noise_ratio"] = df["text"].apply(
        lambda x: (
            round((len(re.findall(r"[^a-zA-Z0-9\sㄱ-ㅎㅏ-ㅣ가-힣]", x)) / len(x)),4)
            if len(x) > 0
            else 0
        )
    )
    return df

In [ ]:
df = calculate_noise_ratio(df)

In [ ]:
less_special_character = df[df["noise_ratio"]<0.1]
print(len(less_special_character))
less_special_character = less_special_character.sort_values(
    by=["noise_ratio"], axis=0, ascending=False
)
less_special_character.to_csv("less_special_character.csv", index=False)
less_special_character.head()

1553


,ID,text,target,hanza,special,filtered_text,noise_ratio
2611,ynat-v1_train_02611,EU 167DI 56w )0 이용DK마크 k9%g최고$프랑c 42% Qq,6,,)%$%,EU 167DI 56w 0 이용DK마크 k9g최고프랑c 42 Qq,0.0976
745,ynat-v1_train_00745,서울 표준주택 공시가 17.75%↑…15억 초과 고가주택 정밀 조준종합2보,4,,.%↑…,서울 표준주택 공시가 177515억 초과 고가주택 정밀 조준종합2보,0.0976
2376,ynat-v1_train_02376,부0 제조업 경기전'지수 곤V)I…조선기자6P 역F 최악,5,,')…,부0 제조업 경기전지수 곤VI조선기자6P 역F 최악,0.0968
126,ynat-v1_train_00126,날씨-미z먼지 보통 수y…오후부터 xJ지고:곳곳 F12일,0,,-…:,날씨미z먼지 보통 수y오후부터 xJ지고곳곳 F12일,0.0968
1974,ynat-v1_train_01974,프As구L수원 5정원 감독 16강 $는 길에 방심=*없다,1,,$=*,프As구L수원 5정원 감독 16강 는 길에 방심없다,0.0968


In [ ]:
high_special_character = df[df["noise_ratio"] >= 0.1]
print(len(high_special_character))
high_special_character = high_special_character.sort_values(
    by=["noise_ratio"], axis=0, ascending=True
)
high_special_character.to_csv("high_special_character.csv", index=False)
high_special_character.head()

1247


,ID,text,target,hanza,special,filtered_text,noise_ratio
1723,ynat-v1_train_01723,"특징X mx소%"" q7적R전망Q1강l",5,,"%""",특징X mx소 q7적R전망Q1강l,0.1
528,ynat-v1_train_00528,朴대M령 북핵은/턱밑 비;M해결 않으면c한미에 큰 피f,2,朴,朴/;,대M령 북핵은턱밑 비M해결 않으면c한미에 큰 피f,0.1
1263,ynat-v1_train_01263,"v남도 P이버O,Z:뉴스에 지역 언론.5포함 O식 요청",3,,",:.",v남도 P이버OZ뉴스에 지역 언론5포함 O식 요청,0.1
2347,ynat-v1_train_02347,北미사일 발사 日언론 신속 보도…NHK는 정규방송 중단,5,北日,北日…,미사일 발사 언론 신속 보도NHK는 정규방송 중단,0.1
2371,ynat-v1_train_02371,현cj* 암fI방K지L하% 보험 출p,5,,*%,현cj 암fI방K지L하 보험 출p,0.1


In [ ]:
# 자주 사용되는 한자와 특수문자 - 노이즈 아닌 것이다.
["美", "中","北", "…", "~", "·", "+", "＋", "∼"]